In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
!pip install "opencv-python-headless<4.3"
import cv2
import xml.etree.ElementTree as ET
import pickle as cPickle
import argparse

import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
import torchvision.transforms as T

import torch.nn as nn
import torch

# !pip install albumentations==1.1.0
!pip install git+https://github.com/albumentations-team/albumentations.git
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import sys
sys.path.insert(0,"/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/dataset")

# from engine import train_one_epoch, evaluate

     |████████████████████████████████| 21.6 MB 6.1 MB/s 
  Cloning https://github.com/albumentations-team/albumentations.git to /tmp/pip-req-build-dk4cz1g8
  Running command git clone -q https://github.com/albumentations-team/albumentations.git /tmp/pip-req-build-dk4cz1g8
  Created wheel for albumentations: filename=albumentations-1.1.0-py3-none-any.whl size=112723 sha256=dbb6bbefd74362f6f82f157cb2114f8d36adea82b1dcd4dc15e5368a2013f537
  Stored in directory: /tmp/pip-ephem-wheel-cache-f_c6_8fc/wheels/09/5b/e0/38cb85186b3cb979092395de69893163f9a223a72066123599
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
# Manage multiple versions of python with pip
# py -3.8 -m pip install package
#https://stackoverflow.com/questions/2812520/dealing-with-multiple-python-versions-and-pip
# Inspired by torchvision example: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

class BirdDataset(torch.utils.data.Dataset):
    """Class to charecterize the bird dataset"""

    def __init__(self, root_dir, transforms=None):
        """
        Args:
            root_dir (string): Directory with all the images
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root = root_dir
        self.transforms = transforms
        
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "all_images")), key=lambda x: int(os.path.splitext(x)[0])))  # list of all image names - jpg
        print(self.imgs)
        self.boxes = list(sorted(os.listdir(os.path.join(root_dir, "all_labels")), key=lambda x: int(os.path.splitext(x)[0]))) # list of all image names - xml
    
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the given index idx"""
        # load images and boxes
        img_path = os.path.join(self.root, "all_images", self.imgs[idx])
        box_path = os.path.join(self.root, "all_labels", self.boxes[idx])
        # print("Image path", img_path)
        # print(type(cv2.imread(img_path, cv2.IMREAD_COLOR)))
        # img = Image.open(img_path).convert("RGB")
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.0
        
        # get boxes for each bird
        document = ET.parse(box_path)
        root = document.getroot()
        boxes = []
        for item in root.findall(".//object/bndbox"):
            xmin = float(item.find('xmin').text)
            xmax = float(item.find('xmax').text)
            ymin = float(item.find('ymin').text)
            ymax = float(item.find('ymax').text)

            box = [xmin, ymin, xmax, ymax]
            boxes.append(box)
        num_objs = len(boxes)

        # convert everything into a torch.Tensor
        image_id = torch.tensor([idx+1])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64) # only one class : a bird
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        # target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        # target["iscrowd"] = iscrowd

        if self.transforms is not None:
            # img = self.transforms(img)
            sample = {
                'image': img,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)

            img = sample['image']
            if len(sample['bboxes']) == 0: # 
                target['boxes'] = torch.zeros((0, 4), dtype=torch.float32)
            else:
              target['boxes'] = torch.tensor(sample['bboxes'])

        return img, target

# class BirdSpeciesDataset():

In [ ]:
def get_model(trained=True, save_path=None):
  device = 'cpu'
  # model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  anchor_sizes = ((32,), (64,), (128,), (256,), (512,))
  aspect_ratios = ((0.25, 0.5, 0.75, 1.0, 2.0, 3.0),) * len(anchor_sizes)
  anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)
  # roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
  #                                               output_size=7,
  #                                               sampling_ratio=2)

  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=True, rpn_anchor_generator=anchor_generator)
  num_classes = 2  # 1 class (bird) + background

  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features

  # replace the pre-trained head with a new one pretrained=False, progress=True, num_classes=91, pretrained_backbone=True, trainable_backbone_layers=None
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
  model.to(device)

  if trained:
    if save_path == None: print("No path to the saved model")
    model.load_state_dict(torch.load(save_path))

  return model

# What is the difference between the YOLO version ?
# def get_YOLOv5model(trained=True, save_path=None):
#   return model

In [ ]:
#########
# UTILS #
#########
def collate_fn(batch):
  """Necessary for DataLoader"""
  return tuple(zip(*batch))

def get_transform(train):
  """Tansform the training and test set. Data Augmentation is made here."""
  transforms = []
  # transforms.append(A.Resize(224, 224, interpolation = cv2.INTER_LANCZOS4))
  if train:
      transforms.append(A.Flip(0.5))
      transforms.append(A.ColorJitter())
      transforms.append(A.RandomCrop(width=1024, height=1024))
      # transforms.append(A.Normalize(mean=[0.5977, 0.5537, 0.5084], std=[0.0899, 0.0806, 0.0759]))           
  transforms.append(ToTensorV2(p=1.0)) 
  return A.Compose(transforms, bbox_params={'format': 'pascal_voc', 'min_visibility': 0.5, 'label_fields': ['labels']})

def save_performance(score, filename):
  """Save the scores (4) on a txt file of name filename"""
  return 0

class Performance():
    """Class to calculate and store the performance/score of a model"""
    def __init__(self, root_save=None, args=None):
        """
        Args:
            root_save (string): 
            params (callable, optional):
        """
        self.root_save = root_save
        self.args = args
        self.train_score = [[],[],[],[]]
        self.validation_score = [[],[],[],[]]
        self.training_iou = []
        self.test_iou = []
        self.iterperepoch = 0
        self.iterperepochval = 0

    def add_score(self, values, training):
      """
      Add the different score of the model for every iteration to the corresponding list of score
      values : dictionnary containing  'loss_classifier', 'loss_box_reg', 'loss_objectness', 'loss_rpn_box_reg'
      """
      if training:
          self.train_score[0].append(values['loss_classifier'].cpu().detach().item())
          self.train_score[1].append(values['loss_box_reg'].cpu().detach().item())
          self.train_score[2].append(values['loss_objectness'].cpu().detach().item())
          self.train_score[3].append(values['loss_rpn_box_reg'].cpu().detach().item())
      else:    
          self.validation_score[0].append(values['loss_classifier'].cpu().detach().item())
          self.validation_score[1].append(values['loss_box_reg'].cpu().detach().item())
          self.validation_score[2].append(values['loss_objectness'].cpu().detach().item())
          self.validation_score[3].append(values['loss_rpn_box_reg'].cpu().detach().item())
    
    def add_accuracy(self, val, training):
        if training:
            self.training_iou.append(val)
        else:
            self.test_iou.append(val)

    def save_class(self, obj):
        """Save the class in txt file"""
        with open(self.root_save, 'wb') as outp:  # Overwrites any existing file.
            cPickle.dump(obj, outp, cPickle.HIGHEST_PROTOCOL)

    def save_score(self, path):
        file = open(path, "w+")
        file.write(str(self.train_score))
        file.close()

    def load(self):
        """Load the class from txt file"""
        file = open(self.root_save,'rb')
        dataPickle = file.read()
        file.close()
        return cPickle.loads(dataPickle)
        # self.__dict__ = cPickle.loads(dataPickle)

def intersect(box_a, box_b):
    """ https://github.com/amdegroot/ssd.pytorch/blob/master/layers/box_utils.py#L48
    We resize both tensors to [A,B,2] without new malloc:
    [A,2] -> [A,1,2] -> [A,B,2]
    [B,2] -> [1,B,2] -> [A,B,2]
    Then we compute the area of intersect between box_a and box_b.
    Args:
      box_a: (tensor) bounding boxes, Shape: [A,4].
      box_b: (tensor) bounding boxes, Shape: [B,4].
    Return:
      (tensor) intersection area, Shape: [A,B].
    """
    A = box_a.size(0)
    B = box_b.size(0)
    max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2))
    min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, :, 0] * inter[:, :, 1]


def jaccard(box_a, box_b):
    """Compute the jaccard overlap of two sets of boxes.  The jaccard overlap
    is simply the intersection over union of two boxes.  Here we operate on
    ground truth boxes and default boxes.
    E.g.:
        A ∩ B / A ∪ B = A ∩ B / (area(A) + area(B) - A ∩ B)
    Args:
        box_a: (tensor) Ground truth bounding boxes, Shape: [num_objects,4]
        box_b: (tensor) Prior boxes from priorbox layers, Shape: [num_priors,4]
    Return:
        jaccard overlap: (tensor) Shape: [box_a.size(0), box_b.size(0)]
    """
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2]-box_b[:, 0]) *
              (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter / union  # [A,B]

def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def batchaccuracy_score(gtboxes, predboxes):
  """ Caculate the accuracy of the predictions over an image based on IOU
      Iteration Over Union
      Args:
        gtboxes: (dict) Ground truth bounding boxes for all images of one batch
        predboxes: (dict) Predicted boxes for all images of one batch (Pytorch output)
  """
  precision = []
  recall = []
  allboxstate = []
  allboxpredscore = []
  for i in range(len(gtboxes)): # per image
    # Filter the bounding boxes with confidence score < 0.5
    predscore = predboxes[i]['scores'].cpu().detach().numpy()
    scoremask = (predscore >= 0.5)
    predscore = predscore[scoremask]
    allboxpredscore += predscore

    # Sort the bounding boxes, first one has highest confidence score
    # orderscoremask = predscore.argsort()[::-1][:len(predscore)]
    # predscore = predscore[orderscoremask]

    # Calculate the IoU score between ground-truth and predictions
    # Check if the order compare to score stay the same
    iouscore = jaccard(gtboxes[i]['boxes'], predscore) # [num_obj, num_prediction]
    iouscore = iouscore.cpu().detach().numpy()

    # 0=FalseNegative, 1=FalsePositive, 2=TruePositive
    for i in range(len(gtboxes[i]['boxes'])):
      # If there is a ground-truth but no prediction: full lign is 0
      if len(iouscore[i]==0) == len(iouscore[i]):
        allboxstate.append(0)
      TPs = len(iouscore[i] >= 0.5) # TPs for one ground-truth, keep only one other are FalsePositive.
      if TPs > 0:
        allboxstate.append(2)
      else:
        allboxstate += [1]*(TPs-1)

      smalliou = iouscore[i][iouscore[i] < 0.5]
      FPs = len(smalliou > 0)
      allboxstate += [1]*(FPs)


In [ ]:
# Global variable
ROOT_DIR_DATA = "/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/dataset"
ROOT_DIR_SAVING = "/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/savedmodel"
SAVEDMODEL_NAME = "/fasterrcnn_SGD0005_Size224_Batch8_Epoch2.pth"
SAVEDPERFORMANCE_NAME = "/score_fasterrcnn_SGD0005_Size224_Batch8_Epoch2.pkl"

# instantiate dataset objects
ds = BirdDataset(ROOT_DIR_DATA, get_transform(train=True))
ds_test = BirdDataset(ROOT_DIR_DATA, get_transform(train=False))

# set hyper-parameters
num_epochs = 1
num_classes = 2
num_coord = 4
num_workers = 2
batch_size = 8

# instantiate data loaders
# split the dataset in train and test set
# random_seed = 1 # or any of your favorite number 
# torch.manual_seed(random_seed)
indices = torch.randperm(len(ds)).tolist()
print(indices)
# torch.manual_seed(random_seed)
# indices = torch.randperm(len(ds)).tolist()
# print(indices)
# indices = torch.randperm(len(ds)).tolist()
# print(indices)
splitdataset = int(len(ds)*0.2)
dataset = torch.utils.data.Subset(ds, indices[:-splitdataset])
dataset_test = torch.utils.data.Subset(ds_test, indices[-splitdataset:])

# define training and validation data loaders
data_loader_training = torch.utils.data.DataLoader(dataset, shuffle=True, collate_fn=collate_fn, num_workers=num_workers, batch_size=batch_size)
data_loader_test = torch.utils.data.DataLoader(dataset_test, shuffle=True, collate_fn=collate_fn, num_workers=num_workers, batch_size=batch_size)

['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg', '51.jpg', '52.jpg', '53.jpg', '54.jpg', '55.jpg', '56.jpg', '57.jpg', '58.jpg', '59.jpg', '60.jpg', '61.jpg', '62.jpg', '63.jpg', '64.jpg', '65.jpg', '66.jpg', '67.jpg', '68.jpg', '69.jpg', '70.jpg', '71.jpg', '72.jpg', '73.jpg', '74.jpg', '75.jpg', '76.jpg', '77.jpg', '78.jpg', '79.jpg', '80.jpg', '81.jpg', '82.jpg', '83.jpg', '84.jpg', '85.jpg', '86.jpg', '87.jpg', '88.jpg', '89.jpg', '90.jpg', '91.jpg', '92.jpg', '93.jpg', '94.jpg', '95.jpg', '96.jpg', '97.jpg', '98.jpg', '99.jpg', '100.jpg', '101.jp

In [ ]:
#################
# Visualisation #
#################
device = "cpu"
images, targets = next(iter(data_loader_training))
# for images, targets in data_loader_training:
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
# print(targets)

for i in range(len(images)):
    # print(targets[i]['boxes'].size())
    boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
    image_id = targets[i]['image_id'].cpu().numpy().astype(np.int32)
    area = targets[i]['area'].cpu().numpy().astype(np.int32)
    print(image_id)
    # print("Area", area)
    # print(images[i].size())
    sample = images[i].permute(1,2,0).cpu().numpy()
    # print(sample)
    # print(sample.shape)
    # print("Box", boxes)

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))
    sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
    for box in boxes:
        cv2.rectangle(sample,
                    (int(box[0]), int(box[1])),
                    (int(box[2]), int(box[3])),
                    (1, 0, 0), 1)
        
    ax.imshow((sample * 255).astype(np.uint8))
    # plt.savefig("/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/"+str(image_id)+"area" + ".png")
    plt.show()

ValueError: ignored

In [ ]:
#################################
# Training and Testing function #
#################################
def training(args):
# ds = BirdDataset(ROOT_DIR_DATA, get_transform(train=True))
  data_loader_training = torch.utils.data.DataLoader(dataset, shuffle=True, collate_fn=collate_fn, num_workers=num_workers, batch_size=args.batch_size)
  data_loader_test = torch.utils.data.DataLoader(dataset_test, shuffle=True, collate_fn=collate_fn, num_workers=num_workers, batch_size=args.batch_size)

  #device = "cpu"
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = get_model(trained=False)

  params = [p for p in model.parameters() if p.requires_grad]
  optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
  # optimizer = torch.optim.Adam(params, lr=0.001)
  # lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
  # lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
  lr_scheduler = None

  itr = 1
  epoch_loss = 0
  perf = Performance(ROOT_DIR_SAVING + SAVEDPERFORMANCE_NAME, args)

  for epoch in range(args.epochs):
      epoch_loss = 0
      iteration = 0
      model.train()

      # Training
      for images, targets in data_loader_training:
          images = list(image.to(device) for image in images)
          targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
          print(targets)

          loss_dict = model(images, targets) # returns losses and detections
          perf.add_score(loss_dict, training=True)
          print("Iter ",str(itr)," - Output model/Loss :", loss_dict)

          losses = sum(loss for loss in loss_dict.values())
          loss_value = losses.item()

          epoch_loss += loss_value

          optimizer.zero_grad()
          losses.backward()
          optimizer.step()

          if itr % 50 == 0:
              print("Iteration" + str(itr) + "loss: " + str(loss_value))
              perf.save_class(perf)
              torch.save(model.state_dict(), ROOT_DIR_SAVING + SAVEDMODEL_NAME)         
              print("SavedOnce")

          itr += 1
          iteration += 1

      # Validation
      itr_val = 0
      for images, targets in data_loader_test:
          images = list(image.to(device) for image in images)
          targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
          # print(images)
          with torch.no_grad():
              loss_dict = model(images, targets) # returns losses and detections
              itr_val += 1
              perf.add_score(loss_dict, training=False)
              print("Validation Loss :", loss_dict)
      
      # Update the learning rate
      if lr_scheduler is not None:
          lr_scheduler.step()

      print("Epoch "+ str(epoch) + "loss: "+ str(epoch_loss/iteration))
      torch.save(model.state_dict(), ROOT_DIR_SAVING + SAVEDMODEL_NAME)
      if perf.iterperepoch == 0:
          perf.iterperepoch = itr
          perf.save_class(perf)
          print("SaveAtEpoch")
      if perf.iterperepochval == 0:
        perf.iterperepoch = itr_val
      # result = train_one_epoch(model, optimizer, data_loader_training, device, epoch, print_freq=10)

  torch.save(model.state_dict(), ROOT_DIR_SAVING + SAVEDMODEL_NAME)
  perf.save_class(perf)
  print("Training is over.")
  print("The model is saved.")

def test(args):
  return 0

In [ ]:
if __name__ == "__main__":
    # Settings
    parser = argparse.ArgumentParser(description='Bird Detection')
    parser.add_argument('-f') #https://stackoverflow.com/questions/42249982/systemexit-2-error-when-calling-parse-args-within-ipython?noredirect=1&lq=1
    parser.add_argument('--eval', type=bool,  default=False, help='Evaluate the model')
    parser.add_argument('--model', type=str, default='fasterrcnn', metavar='N',
                        choices=['fasterrcnn'], help='Model to use')
    parser.add_argument('--batch_size', type=int, default=8, metavar='batch_size',
                        help='Size of batch)')
    parser.add_argument('--epochs', type=int, default=20, metavar='N',
                        help='Number of episode to train ')
    parser.add_argument('--use_sgd', type=bool, default=True,
                        help='Use SGD')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                        help='learning rate (default: 0.001, 0.1 if using sgd)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                        help='SGD momentum (default: 0.9)')
    parser.add_argument('--scheduler', type=str, default='cos', metavar='N',
                        choices=['cos', 'step'],
                        help='Scheduler to use, [cos, step]')

    args = parser.parse_args()

    if not args.eval:
      training(args)
    else:
      test(args)
    

[{'boxes': tensor([]), 'labels': tensor([1, 1, 1, 1, 1]), 'image_id': tensor([651]), 'area': tensor([84357., 96000., 52355., 23652., 30179.])}, {'boxes': tensor([]), 'labels': tensor([1, 1, 1, 1, 1, 1, 1]), 'image_id': tensor([141]), 'area': tensor([31104., 28305., 33972., 36340., 65824., 24300., 40446.])}, {'boxes': tensor([]), 'labels': tensor([1, 1, 1, 1, 1, 1]), 'image_id': tensor([288]), 'area': tensor([ 6426.,  7772., 10965.,  7482.,  9928.,  8418.])}, {'boxes': tensor([]), 'labels': tensor([1, 1]), 'image_id': tensor([224]), 'area': tensor([ 99603., 100084.])}, {'boxes': tensor([]), 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'image_id': tensor([624]), 'area': tensor([6592., 5883., 2262., 5888., 4176., 4902., 6956., 3663., 2821., 4779.,
        7524., 4929., 3384., 3650.])}, {'boxes': tensor([]), 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'image_id': tensor([450]), 'area': tensor([6716., 4292., 4224., 4816., 5412., 6300., 4200., 5467., 5135., 4002.

ValueError: ignored

In [ ]:
!pip3 install pickle5
import pickle5 as cPickle

     |████████████████████████████████| 256 kB 7.9 MB/s 


In [ ]:
path = "/content/drive/MyDrive/Thesis/savedmodel/score_fasterrcnn_SGD0005_SchedulerNone_Size576_Batch8_Epoch50_8020_anchor.pkl"
perf1 = Performance(root_save=path)
perf1 = perf1.load()
# Plot the score of training and validation dataset of Faster RCNN
# 
print(perf1.iterperepoch)
print(perf1.iterperepochval)
print(len(perf1.train_score[0]))
print(len(perf1.validation_score[0]))
print(perf1.args)

67
17
3300
850
Namespace(batch_size=8, epochs=50, eval=False, f=None, lr=0.001, model='fasterrcnn', momentum=0.9, scheduler='None', use_sgd=True)


In [ ]:
train_score = perf1.train_score
val_score = perf1.validation_score

# Mean for epoch of training score
sizepoch = perf1.iterperepoch-1
epochscore1 = []
sum = 0
for i in range(len(train_score[0])):
  sum += train_score[0][i]
  if ((i+1) % sizepoch) == 0:
    epochscore1.append(sum/sizepoch)
    sum = 0
epochscore2 = []
sum = 0
for i in range(len(train_score[0])):
  sum += train_score[1][i]
  if ((i+1) % sizepoch) == 0:
    epochscore2.append(sum/sizepoch)
    sum = 0
epochscore3 = []
sum = 0
for i in range(len(train_score[0])):
  sum += train_score[2][i]
  if ((i+1) % sizepoch) == 0:
    epochscore3.append(sum/sizepoch)
    sum = 0
epochscore4 = []
sum = 0
for i in range(len(train_score[0])):
  sum += train_score[3][i]
  if ((i+1) % sizepoch) == 0:
    epochscore4.append(sum/sizepoch)
    sum = 0

# Mean for epoch of validation score
sizepoch = perf1.iterperepochval
epochvalscore1 = []
sum = 0
for i in range(len(val_score[0])):
  sum += val_score[0][i]
  if ((i+1) % sizepoch) == 0:
    epochvalscore1.append(sum/sizepoch)
    sum = 0
epochvalscore2 = []
sum = 0
for i in range(len(val_score[0])):
  sum += val_score[1][i]
  if ((i+1) % sizepoch) == 0:
    epochvalscore2.append(sum/sizepoch)
    sum = 0
epochvalscore3 = []
sum = 0
for i in range(len(val_score[0])):
  sum += val_score[2][i]
  if ((i+1) % sizepoch) == 0:
    epochvalscore3.append(sum/sizepoch)
    sum = 0
epochvalscore4 = []
sum = 0
for i in range(len(val_score[0])):
  sum += val_score[3][i]
  if ((i+1) % sizepoch) == 0:
    epochvalscore4.append(sum/sizepoch)
    sum = 0

print(len(epochscore1))

fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(15, 6))
# fig, axs = plt.subplots(nrows=2,ncols=3, sharex=True, sharey=True)
fig.suptitle('Score')
# fig.tight_layout()
# peut-être mieux: plt.subplots_adjust

X = range(len(epochscore1)) + np.ones(len(epochscore1))
Xval = range(len(epochvalscore1))+ np.ones(len(epochscore1))

axs[0,0].plot(X, epochscore1, label="Training", color='C0')
axs[0,0].scatter(X, epochscore1, color='C0')
axs[0,0].plot(Xval, epochvalscore1, color='C1', label="Validation")
axs[0,0].scatter(Xval, epochvalscore1, color='C1')
#axs[0,0].set_title('Sample 1')
axs[0,0].set_ylabel('Loss classifier')
# axs[0,0].set_xlabel('x1')
axs[0,0].legend()

axs[0,1].plot(X, epochscore2, color='C0', label="Training")
axs[0,1].scatter(X, epochscore2, color='C0')
axs[0,1].plot(Xval, epochvalscore2, color='C1', label="Validation")
axs[0,1].scatter(Xval, epochvalscore2, color='C1')
#axs[0,1].set_title('Sample 2')
axs[0,1].set_ylabel('Loss box reg')
axs[0,1].legend()
#axs[0,1].set_xlabel('x2')

# axs[1,0].scatter(X, train_score[2], color = "blue", edgecolors = "white", linewidths = 0.1, s=20, alpha = 0.7)
axs[1,0].plot(X, epochscore3, color='C0', label="Training")
axs[1,0].scatter(X, epochscore3, color='C0')
axs[1,0].plot(Xval, epochvalscore3, color='C1', label="Validation")
axs[1,0].scatter(Xval, epochvalscore3, color='C1')
#axs[1,0].set_title('Sample 3')
axs[1,0].set_ylabel('Loss objectness')
axs[1,0].set_xlabel('Epoch')
axs[1,0].legend()

axs[1,1].plot(X, epochscore4, color='C0', label="Training")
axs[1,1].scatter(X, epochscore4, color='C0')
axs[1,1].plot(Xval, epochvalscore4, color='C1', label="Validation")
axs[1,1].scatter(Xval, epochvalscore4, color='C1')
#axs[1,1].set_title('Sample 4')
axs[1,1].set_ylabel('Loss rpn_box_reg')
axs[1,1].set_xlabel('Epoch')
axs[1,1].legend()

for subg in axs.flat: # Apply settings on all subgraphs
    subg.set_facecolor('white') # remove grey background

    subg.spines['right'].set_visible(False) # Hide the right and top spines
    subg.spines['top'].set_visible(False) 
    subg.yaxis.set_ticks_position('left') # only show left and bottom axis
    subg.xaxis.set_ticks_position('bottom')

plt.rcParams["figure.figsize"] = (40,20) # remove to see overlapping subplots
# plt.savefig("/content/drive/MyDrive/Github/BirdDetection-FeaturesExtraction/images/performance/score_Batch8.png")
plt.show()

NameError: ignored

In [ ]:
path1 = "/content/drive/MyDrive/Thesis/savedmodel/WithCOCO/score_fasterrcnn_SGD0005_SchedulerNone_Size224_Batch8_Epoch25_8020.pkl"
perf1 = Performance(root_save=path1)
perf1 = perf1.load()
# path1 = "/content/drive/MyDrive/Thesis/savedmodel/score_fasterrcnn_SGD0005_SchedulerNone_Size576_Batch8_Epoch50_8020_anchor.pkl"
# perf1 = Performance(root_save=path1)
# perf1 = perf1.load()
path2 = "/content/drive/MyDrive/Thesis/savedmodel/score_fasterrcnn_SGD0005_SchedulerNone_Size576_Batch8_Epoch50_8020.pkl"
perf2 = Performance(root_save=path2)
perf2 = perf2.load()


fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(15, 6))
# fig, axs = plt.subplots(nrows=2,ncols=3, sharex=True, sharey=True)
fig.suptitle('Score')
# fig.tight_layout()
# peut-être mieux: plt.subplots_adjust

def add_performance(perf, color):
  train_score = perf.train_score
  val_score = perf.validation_score

  # Mean for epoch of training score
  sizepoch = perf.iterperepoch-1
  epochscore1 = []
  sum = 0
  for i in range(len(train_score[0])):
    sum += train_score[0][i]
    if ((i+1) % sizepoch) == 0:
      epochscore1.append(sum/sizepoch)
      sum = 0
  epochscore2 = []
  sum = 0
  for i in range(len(train_score[0])):
    sum += train_score[1][i]
    if ((i+1) % sizepoch) == 0:
      epochscore2.append(sum/sizepoch)
      sum = 0
  epochscore3 = []
  sum = 0
  for i in range(len(train_score[0])):
    sum += train_score[2][i]
    if ((i+1) % sizepoch) == 0:
      epochscore3.append(sum/sizepoch)
      sum = 0
  epochscore4 = []
  sum = 0
  for i in range(len(train_score[0])):
    sum += train_score[3][i]
    if ((i+1) % sizepoch) == 0:
      epochscore4.append(sum/sizepoch)
      sum = 0

  # Mean for epoch of validation score
  sizepoch = perf.iterperepochval
  epochvalscore1 = []
  sum = 0
  for i in range(len(val_score[0])):
    sum += val_score[0][i]
    if ((i+1) % sizepoch) == 0:
      epochvalscore1.append(sum/sizepoch)
      sum = 0
  epochvalscore2 = []
  sum = 0
  for i in range(len(val_score[0])):
    sum += val_score[1][i]
    if ((i+1) % sizepoch) == 0:
      epochvalscore2.append(sum/sizepoch)
      sum = 0
  epochvalscore3 = []
  sum = 0
  for i in range(len(val_score[0])):
    sum += val_score[2][i]
    if ((i+1) % sizepoch) == 0:
      epochvalscore3.append(sum/sizepoch)
      sum = 0
  epochvalscore4 = []
  sum = 0
  for i in range(len(val_score[0])):
    sum += val_score[3][i]
    if ((i+1) % sizepoch) == 0:
      epochvalscore4.append(sum/sizepoch)
      sum = 0

  print(len(epochscore1))

  X = range(len(epochscore1)) + np.ones(len(epochscore1))
  Xval = range(len(epochvalscore1))+ np.ones(len(epochscore1))

  axs[0,0].plot(X, epochscore1, label="Training", color=color[0])
  axs[0,0].scatter(X, epochscore1, color=color[0])
  axs[0,0].plot(Xval, epochvalscore1, color=color[1], label="Validation")
  axs[0,0].scatter(Xval, epochvalscore1, color=color[1])

  axs[0,1].plot(X, epochscore2, color=color[0], label="Training")
  axs[0,1].scatter(X, epochscore2, color=color[0])
  axs[0,1].plot(Xval, epochvalscore2, color=color[1], label="Validation")
  axs[0,1].scatter(Xval, epochvalscore2, color=color[1])
  #axs[0,1].set_xlabel('x2')

  # axs[1,0].scatter(X, train_score[2], color = "blue", edgecolors = "white", linewidths = 0.1, s=20, alpha = 0.7)
  axs[1,0].plot(X, epochscore3, color=color[0], label="Training")
  axs[1,0].scatter(X, epochscore3, color=color[0])
  axs[1,0].plot(Xval, epochvalscore3, color=color[1], label="Validation")
  axs[1,0].scatter(Xval, epochvalscore3, color=color[1])
  #axs[1,0].set_title('Sample 3')

  axs[1,1].plot(X, epochscore4, color=color[0], label="Training")
  axs[1,1].scatter(X, epochscore4, color=color[0])
  axs[1,1].plot(Xval, epochvalscore4, color=color[1], label="Validation")
  axs[1,1].scatter(Xval, epochvalscore4, color=color[1])
  #axs[1,1].set_title('Sample 4')

add_performance(perf1, ['C0', 'C1'])
add_performance(perf2, ['C2', 'C3'])

#axs[0,0].set_title('Sample 1')
axs[0,0].set_ylabel('Loss classifier')
# axs[0,0].set_xlabel('x1')
axs[0,0].legend()
#axs[0,1].set_title('Sample 2')
axs[0,1].set_ylabel('Loss box reg')
axs[0,1].legend()
axs[1,0].set_ylabel('Loss objectness')
axs[1,0].set_xlabel('Epoch')
axs[1,0].legend()
axs[1,1].set_ylabel('Loss rpn_box_reg')
axs[1,1].set_xlabel('Epoch')
axs[1,1].legend()

In [5]:
!pip install CocoDataset==0.1.2

In [29]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# !wget --spider --recursive http://images.cocodataset.org
!unzip /content/annotations_trainval2017.zip

--2022-03-14 09:38:52--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.200.67
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.200.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  81.0MB/s    in 3.0s    

2022-03-14 09:38:56 (81.0 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  /content/annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


In [ ]:
from coco_dataset import coco_dataset_download as cocod
class_name='person'  #class name example 
images_count=42       #count of images  
annotations_path='/content/annotations/instances_val2017.json' #path of coco dataset annotations 
#call download function 
cocod.coco_dataset_download(class_name,images_count,annotations_path)

In [34]:
import shutil
shutil.rmtree('/content/annotations')